In [14]:
#Importing necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from google.colab import drive

In [17]:
# Loading the dataset

drive.mount('/content/drive')
df_train = pd.read_csv('/content/drive/MyDrive/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/test.csv')
df_train.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [18]:
# Function to clean the text data
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Convert to lowercase
    text = text.lower().strip()
    return text

# Cleaning the text column in the dataset
df_train['text_clean'] = df_train['text'].apply(clean_text)

In [20]:
# Performing TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(df_train['text_clean']).toarray()

In [21]:
# Splitting the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_tfidf,
    df_train['target'],
    test_size=0.2,
    random_state=42
)

In [22]:
# Exploratory Data Analysis

# Class distribution
class_distribution = df_train['target'].value_counts(normalize=True)

# Top words in the dataset
top_words = pd.DataFrame(
    X_tfidf.sum(axis=0),
    index=tfidf_vectorizer.get_feature_names_out(),
    columns=['tfidf']
).sort_values(by='tfidf', ascending=False).head(20)

print("Class Distribution:")
print(class_distribution)

print("\nTop 10 Words:")
print(top_words.head(10))

Class Distribution:
0    0.57034
1    0.42966
Name: target, dtype: float64

Top 10 Words:
              tfidf
im        84.939923
like      83.297119
just      77.636830
amp       65.168897
dont      56.341773
new       53.049353
people    49.995554
news      45.960922
disaster  43.203076
video     42.401831
